In [3]:
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as rn
import time 
from sklearn.decomposition import RandomizedPCA
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from numpy import linalg as LA
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import chi2_kernel
from scipy import linalg
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, pairwise
from sklearn.cluster import KMeans
from scipy.io import loadmat
from collections import Counter
from scipy.interpolate import Rbf
from sklearn.decomposition import PCA, KernelPCA

In [4]:
import json
with open('feature_data.json', 'r') as f:
    features = json.load(f)

features_ar = np.array(features)
print(len(features_ar))

14096


In [5]:
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['train_idx'].flatten()
camIds = loadmat('cuhk03_new_protocol_config_labeled.mat')['camId'].flatten()
filelists = loadmat('cuhk03_new_protocol_config_labeled.mat')['filelist'].flatten()
gallery_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['gallery_idx'].flatten()
labels = loadmat('cuhk03_new_protocol_config_labeled.mat')['labels'].flatten()
query_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['query_idx'].flatten()

train_idxss = np.subtract(train_idxs, 1)
gallery_idxss = np.subtract(gallery_idxs, 1)
query_idxss = np.subtract(query_idxs, 1)

In [6]:
camIds_train = camIds[train_idxss]
filelists_train = filelists[train_idxss]
labels_train = labels[train_idxss]
features_train = features_ar[train_idxss]

camIds_gal = camIds[gallery_idxss]
filelists_gal = filelists[gallery_idxss]
labels_gal = labels[gallery_idxss]
features_gal = features_ar[gallery_idxss]

camIds_query = camIds[query_idxss]
filelists_query = filelists[query_idxss]
labels_query = labels[query_idxss]
features_query = features_ar[query_idxss]

In [7]:
def scores_query(ranklist, query_l, instances, k):
    acc = 0
    TP = 0
    FP = 0
    prec = 0
    rec = 0

    if rankKeval(ranklist, query_l) == True:
        acc = 1
        
    for i in range (0, k):
        if (ranklist[i] == query_l):
            TP = TP + 1
        else:
            FP = FP +1
    prec = TP/(TP + FP)
    rec = TP/instances
    
    return(acc, prec, rec)

In [8]:
def rankKeval(labels_neigh, label_query):
    contained = False
    for i in range (0, len(labels_neigh)):
        if (labels_neigh[i] == label_query):
            contained = True
    return(contained)

In [12]:
def kNNchikernel(features_gal, labels_gal, camIds_gal, features_query, labels_query, camIds_query, k):
    acc = 0
    n_query = len(labels_query)
    n_gal = len(labels_gal)
    rbf_gq = pairwise.chi2_kernel(features_query, features_gal)
    rbf_gg = pairwise.chi2_kernel(features_gal, features_gal)
    rbf_qq = pairwise.chi2_kernel(features_query, features_query)
    rbfm = np.empty([n_query, n_gal])
    for m in range (0, n_query):
        for z in range (0, n_gal):
            rbfm[m][z] = rbf_gg[z][z] + rbf_qq[m][m] - 2*rbf_gq[m][z]
    for i in range(0, n_query):
        dist = np.copy(rbfm[[i], :])[0]
        ids = np.copy(camIds_gal)
        lbs = np.copy(labels_gal)
        inds = list(range(0, n_gal))
        neigh = []
        while len(neigh)<k:
            rem = False
            mind = np.argmin(dist)
            ind = inds[mind]
            if ((ids[mind] == camIds_query[i]) and (lbs[mind] == labels_query[i])):
                rem = True
            if rem == False:
                neigh = np.append(neigh, [lbs[mind]], axis=0)
            dist = np.delete(dist, mind, axis=0)
            inds = np.delete(inds, mind, axis=0)
            ids = np.delete(ids, mind, axis=0)
            lbs = np.delete(lbs, mind, axis=0)

        accq, precq, recq = scores_query(neigh, labels_query[i], 1, k) # CHANGE INSTANCES HERE
        acc = acc + accq
    acc = acc/n_query
    print('acc = ', acc)
    
    return(acc)

In [13]:
acc = kNNchikernel(features_gal, labels_gal, camIds_gal, features_query, labels_query, camIds_query, 1)

acc =  0.0014285714285714286


In [9]:
def kNNcosskernel(features_gal, labels_gal, camIds_gal, features_query, labels_query, camIds_query, k):
    acc = 0
    n_query = len(labels_query)
    n_gal = len(labels_gal)
    rbf_gq = pairwise.cosine_similarity(features_query, features_gal)
    rbf_gg = pairwise.cosine_similarity(features_gal, features_gal)
    rbf_qq = pairwise.cosine_similarity(features_query, features_query)
    rbfm = np.empty([n_query, n_gal])
    for m in range (0, n_query):
        for z in range (0, n_gal):
            rbfm[m][z] = rbf_gg[z][z] + rbf_qq[m][m] - 2*rbf_gq[m][z]
    for i in range(0, n_query):
        dist = np.copy(rbfm[[i], :])[0]
        ids = np.copy(camIds_gal)
        lbs = np.copy(labels_gal)
        inds = list(range(0, n_gal))
        neigh = []
        while len(neigh)<k:
            rem = False
            mind = np.argmin(dist)
            ind = inds[mind]
            if ((ids[mind] == camIds_query[i]) and (lbs[mind] == labels_query[i])):
                rem = True
            if rem == False:
                neigh = np.append(neigh, [lbs[mind]], axis=0)
            dist = np.delete(dist, mind, axis=0)
            inds = np.delete(inds, mind, axis=0)
            ids = np.delete(ids, mind, axis=0)
            lbs = np.delete(lbs, mind, axis=0)

        accq, precq, recq = scores_query(neigh, labels_query[i], 1, k) # CHANGE INSTANCES HERE
        acc = acc + accq
    acc = acc/n_query
    print('acc = ', acc)
    
    return(acc)

In [10]:
def kNNcosdkernel(features_gal, labels_gal, camIds_gal, features_query, labels_query, camIds_query, k):
    acc = 0
    n_query = len(labels_query)
    n_gal = len(labels_gal)
    rbf_gq = pairwise.cosine_distances(features_query, features_gal)
    rbf_gg = pairwise.cosine_distances(features_gal, features_gal)
    rbf_qq = pairwise.cosine_distances(features_query, features_query)
    rbfm = np.empty([n_query, n_gal])
    for m in range (0, n_query):
        for z in range (0, n_gal):
            rbfm[m][z] = rbf_gg[z][z] + rbf_qq[m][m] - 2*rbf_gq[m][z]
    for i in range(0, n_query):
        dist = np.copy(rbfm[[i], :])[0]
        ids = np.copy(camIds_gal)
        lbs = np.copy(labels_gal)
        inds = list(range(0, n_gal))
        neigh = []
        while len(neigh)<k:
            rem = False
            mind = np.argmin(dist)
            ind = inds[mind]
            if ((ids[mind] == camIds_query[i]) and (lbs[mind] == labels_query[i])):
                rem = True
            if rem == False:
                neigh = np.append(neigh, [lbs[mind]], axis=0)
            dist = np.delete(dist, mind, axis=0)
            inds = np.delete(inds, mind, axis=0)
            ids = np.delete(ids, mind, axis=0)
            lbs = np.delete(lbs, mind, axis=0)

        accq, precq, recq = scores_query(neigh, labels_query[i], 1, k) # CHANGE INSTANCES HERE
        acc = acc + accq
    acc = acc/n_query
    print('acc = ', acc)
    
    return(acc)

In [11]:
acc = kNNcosskernel(features_gal, labels_gal, camIds_gal, features_query, labels_query, camIds_query, 1)
acc = kNNcosdkernel(features_gal, labels_gal, camIds_gal, features_query, labels_query, camIds_query, 1)

acc =  0.4757142857142857
acc =  0.0


In [9]:
def do_kNN(features_gal, labels_gal, camIds_gal, features_query, labels_query, camIds_query, k, s_net, instances):
    knn = build_kNN(features_gal, labels_gal, k, s_net)
    query_n = len(labels_query)
    print(query_n)
    acc = [0] * (query_n * k)
    prec = [0] * (query_n * k)
    rec = [0] * (query_n * k)
    AA = [0] * k
    AR = [0] * k 
    AP = [0] * k
    F1 = [0] * k
    mAP = [0] * k 
    for i in range (0, query_n):
        query_l = labels_query[i]
        query_f = features_query[i]
        query_Id = camIds_query[i]
        
        
        neigh = ret_NN(knn, query_l, query_f, query_Id, k, s_net, camIds_gal)
        
        
        for z in range (0, k):
            ind = i+(z*query_n)
            acc[ind], prec[ind], rec[ind] = scores_query(neigh, query_l, instances[i], (k-z))

            neigh.pop()
    for i in range (0, k):
        LB = i*query_n
        UB = (i+1)*query_n
        AA[i], AP[i], AR[i] = scores_k(acc[LB:UB], prec[LB:UB], rec[LB:UB], i+1, query_n)
        F1[i] = 2*(AP[i][0]*AR[i][0])/(AP[i][0]+AR[i][0])
        print(sum(AP[0]))
    mAP[i] = 1
    return(AA, AR, AP, mAP, F1)

In [10]:
acc = kNNmanhkernel(features_gal, labels_gal, camIds_gal, features_query, labels_query, camIds_query, 1)

acc =  0.47214285714285714


In [10]:
acc = kNNcosskernel(features_gal, labels_gal, camIds_gal, features_query, labels_query, camIds_query, 20)

acc =  0.8271428571428572


In [14]:
features_g, features_q, labels_g, labels_q, camIds_g, camIds_q = train_test_split(features_train, labels_train, camIds_train, test_size=0.33, stratify=labels_train)
acc = kNNcosskernel(features_g, labels_g, camIds_g, features_q, labels_q, camIds_q, 1)

acc =  0.9958881578947368


In [ ]:
kpca = KernelPCA(kernel="rbf", fit_inverse_transform=True, gamma=10)
X_kpca = kpca.fit_transform(X)


In [1]:
def kNN_find(features_gal, labels_gal, camIds_gal, features_query, labels_query, camIds_query, k, tr_pairs):
    acc = 0
    n_query = len(labels_query)
    n_gal = len(labels_gal)
    for i in range(0, n_query):
        dist[i] = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])        
        ids = np.copy(camIds_gal)
        lbs = np.copy(labels_gal)
        inds = list(range(0, n_gal))
        neigh = []
        while len(neigh)<k:
            rem = False
            mind = np.argmin(dist)
            ind = inds[mind]
            if ((ids[mind] == camIds_query[i]) and (lbs[mind] == labels_query[i])):
                rem = True
            if rem == False:
                neigh = np.append(neigh, [lbs[mind]], axis=0)
            dist = np.delete(dist, mind, axis=0)
            inds = np.delete(inds, mind, axis=0)
            ids = np.delete(ids, mind, axis=0)
            lbs = np.delete(lbs, mind, axis=0)

        accq, precq, recq = scores_query(neigh, labels_query[i], 1, k) # CHANGE INSTANCES HERE
        acc = acc + accq
    acc = acc/n_query
    print('acc = ', acc)
    
    return(acc)

In [1]:
def do_kNN(features_gal, labels_gal, camIds_gal, features_query, labels_query, camIds_query, k, s_net, instances):
    knn = build_kNN(features_gal, labels_gal, k, s_net)
    query_n = len(labels_query)
    print(query_n)
    acc = [0] * (query_n * k)
    prec = [0] * (query_n * k)
    rec = [0] * (query_n * k)
    AA = [0] * k
    AR = [0] * k 
    AP = [0] * k
    F1 = [0] * k
    mAP = [0] * k 
    for i in range (0, query_n):
        query_l = labels_query[i]
        query_f = features_query[i]
        query_Id = camIds_query[i]
        dist = model.predict([big_query(features_query, i, len(features_gal)).reshape(-1,52,86), features_gal.reshape(-1,52,86)])
        
        ids = np.copy(camIds_gal)
        lbs = np.copy(labels_gal)
        inds = list(range(0, n_gal))
        neigh = []
        while len(neigh)<k:
            rem = False
            mind = np.argmin(dist)
            ind = inds[mind]
            if ((ids[mind] == camIds_query[i]) and (lbs[mind] == labels_query[i])):
                rem = True
            if rem == False:
                neigh = np.append(neigh, [lbs[mind]], axis=0)
            dist = np.delete(dist, mind, axis=0)
            inds = np.delete(inds, mind, axis=0)
            ids = np.delete(ids, mind, axis=0)
            lbs = np.delete(lbs, mind, axis=0)
            
        for z in range (0, k):
            ind = i+(z*query_n)
            acc[ind], prec[ind], rec[ind] = scores_query(neigh, query_l, instances[i], (k-z))

            neigh.pop()
    for i in range (0, k):
        LB = i*query_n
        UB = (i+1)*query_n
        AA[i], AP[i], AR[i] = scores_k(acc[LB:UB], prec[LB:UB], rec[LB:UB], i+1, query_n)
        F1[i] = 2*(AP[i][0]*AR[i][0])/(AP[i][0]+AR[i][0])
        print(sum(AP[0]))
    mAP[i] = 1
    return(AA, AR, AP, mAP, F1)

In [ ]:
def kNNmanhkernel(features_gal, labels_gal, camIds_gal, features_query, labels_query, camIds_query, k):
    acc = 0
    n_query = len(labels_query)
    n_gal = len(labels_gal)
    rbfm = pairwise.manhattan_distances(features_query, features_gal)
#     rbf_gg = pairwise.manhattan_distances(features_gal, features_gal)
#     rbf_qq = pairwise.manhattan_distances(features_query, features_query)
#     rbfm = np.empty([n_query, n_gal])
#     for m in range (0, n_query):
#         for z in range (0, n_gal):
#             rbfm[m][z] = rbf_gg[z][z] + rbf_qq[m][m] - 2*rbf_gq[m][z]
    for i in range(0, n_query):
        dist = np.copy(rbfm[[i], :])[0]
        ids = np.copy(camIds_gal)
        lbs = np.copy(labels_gal)
        inds = list(range(0, n_gal))
        neigh = []
        while len(neigh)<k:
            rem = False
            mind = np.argmin(dist)
            ind = inds[mind]
            if ((ids[mind] == camIds_query[i]) and (lbs[mind] == labels_query[i])):
                rem = True
            if rem == False:
                neigh = np.append(neigh, [lbs[mind]], axis=0)
            dist = np.delete(dist, mind, axis=0)
            inds = np.delete(inds, mind, axis=0)
            ids = np.delete(ids, mind, axis=0)
            lbs = np.delete(lbs, mind, axis=0)

        accq, precq, recq = scores_query(neigh, labels_query[i], 1, k) # CHANGE INSTANCES HERE
        acc = acc + accq
    acc = acc/n_query
    print('acc = ', acc)
    
    return(acc)